In [241]:
import config

import os
import numpy as np
import pandas as pd
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

from time import sleep
from random import randint

In [17]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [2]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [29]:
# https://open.spotify.com/playlist/37i9dQZF1DX2TRYkJECvfC

track_list = get_playlist_tracks("spotify", "37i9dQZF1DX2TRYkJECvfC")


In [42]:
song_audio_features.shape

(200, 18)

In [40]:
## build dataframe of info
list_of_songs=[]
for index in range(len(track_list)):
    list_of_songs.append(sp.audio_features(track_list[index]['track']['id'])[0])
song_audio_features = pd.DataFrame(list_of_songs) 

In [120]:
def save_features_csv_for_all_tracks_in_playlist(playlist_id):
    
    outdir = os.path.join('Data', 'audio_features')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    outfilename = os.path.join(outdir, f'{playlist_id}.csv')
    
    track_list = get_playlist_tracks("spotify", playlist_id)
    
    list_of_songs=[]
    for i in range(len(track_list)):
        features = sp.audio_features(track_list[i]['track']['id'])[0]
        if features is None:
            print(f"No audio features available for track: {track_list[i]['track']['id']}")
            continue
        else:
            list_of_songs.append(features)
            
    song_audio_features = pd.DataFrame(list_of_songs)
    song_audio_features.to_csv(outfilename, index=False)
    
    print(f'Saved {len(list_of_songs)} tracks in file: {outfilename}')
    return
    

In [219]:
indir = 'Data/audio_features'
infile_list = os.listdir(indir)

In [240]:
testfile = infile_list[0]
print(f'Processing file: {testfile}')

save_features_csv_for_all_tracks_in_albums(os.path.join(indir, testfile))

Processing file: 37i9dQZF1DX873GaRGUmPl.csv
Saved 835 tracks in file: Data/audio_features2/37i9dQZF1DX873GaRGUmPl.csv


In [243]:
for filename in infile_list[1:]:
    print(f'Processing file: {filename}')
    save_features_csv_for_all_tracks_in_albums(os.path.join(indir, filename))


Processing file: timeout_the-50-best-80s-songs.csv
Saved 821 tracks in file: Data/audio_features2/timeout_the-50-best-80s-songs.csv
Processing file: 37i9dQZF1DXaLI8a395lse.csv
Saved 1316 tracks in file: Data/audio_features2/37i9dQZF1DXaLI8a395lse.csv
Processing file: 37i9dQZF1DXbwcS5W73XRr.csv
Saved 601 tracks in file: Data/audio_features2/37i9dQZF1DXbwcS5W73XRr.csv
Processing file: timeout_best-pop-songs-of-all-time.csv
Saved 601 tracks in file: Data/audio_features2/timeout_best-pop-songs-of-all-time.csv
Processing file: 37i9dQZF1DX00RdhV73Dbe.csv
Saved 421 tracks in file: Data/audio_features2/37i9dQZF1DX00RdhV73Dbe.csv
Processing file: 37i9dQZF1DWSlskcBvOTt0.csv
Saved 412 tracks in file: Data/audio_features2/37i9dQZF1DWSlskcBvOTt0.csv
Processing file: 37i9dQZF1DX1MUPbVKMgJE.csv
Saved 605 tracks in file: Data/audio_features2/37i9dQZF1DX1MUPbVKMgJE.csv
Processing file: 37i9dQZF1DXarRysLJmuju.csv
Saved 515 tracks in file: Data/audio_features2/37i9dQZF1DXarRysLJmuju.csv
Processing file: 

In [242]:
def save_features_csv_for_all_tracks_in_albums(infilepath):
    
    # extract the name of the input file, without the csv suffix
    songlistname = '.'.join(os.path.basename(infilepath).split('.')[:-1])
    
    outdir = os.path.join('Data', 'audio_features2')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    outfilename = os.path.join(outdir, f'{songlistname}.csv')
    
    my_tracks = pd.read_csv(infilepath, index_col=None)

    list_of_songs=[]
    for i in range(my_tracks.shape[0]):
        sleep(randint(1,3))
        # get album for this track
        track = sp.track(my_tracks.id[i], market='GB')
        album_tracks = sp.album_tracks(track['album']['id'])
        track_features = sp.audio_features([album_tracks['items'][i]['id'] for i in range(len(album_tracks['items']))])
        list_of_songs += [x for x in track_features if x is not None]

    song_audio_features = pd.DataFrame(list_of_songs)
    song_audio_features.to_csv(outfilename, index=False)
    
    print(f'Saved {len(list_of_songs)} tracks in file: {outfilename}')
    return

In [180]:
sp.audio_features('sss')

[None]

In [152]:
# list of albums from this artist
artist_albums = sp.artist_albums('2WBJQGf1bT1kxuoqziH5g4', country='GB', album_type='album') #, country=None, limit=20, offset=0)

In [162]:
# get tracks on this album (iter i=0)
album_tracks = sp.album_tracks(artist_albums['items'][0]['id'])

In [179]:
album_tracks['items'][0]['id']

'5VqjJgqiHFKidCnxpxkVjM'

In [ ]:
# get audio features for track
feat = sp.audio_features()[0]

In [18]:
def get_artists_from_track(track):
    return [artist["name"] for artist in track["artists"]]

def get_artists_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    return list(set(artist for subset in [get_artists_from_track(track["track"]) for track in tracks_from_playlist] for artist in subset))

In [136]:
def get_artist_ids_from_track(track):
    return [artist["id"] for artist in track["artists"]]

In [ ]:
def get_all_tracks_from_artists(fname):
    df = pd.read_csv(f'Data/audio_features/{fname}.csv')
    
    for i in df.shape[0]:
        trackinfo = sp.track(df.id[i], market='GB')
        artist_ids = get_artist_ids_from_track(trackinfo)
        
        

In [208]:
def display_artist(track_info):
## create a single string with all artist names
    return " / ".join([x['name'] for x in track_info['artists']])

## Get artist and track title

In [245]:
## load track ids from csvs with audio features
indir = 'Data/audio_features/'
infile_list = os.listdir(indir)

dfs = []
for fname in infile_list:
    fname_tracks = pd.read_csv(os.path.join(indir, fname), index_col=None)
    dfs.append(fname_tracks.id)
    
track_ids = pd.concat(dfs, axis=0)
track_ids = track_ids.drop_duplicates().reset_index(drop=True)

# GET artist <<<<<<<<<<----------

In [248]:
def get_artist_etc_given_id(track_ids):
    ## use the ids to get the title (name) and artist info for each track
    titles=[]
    artists=[]
    pp_artist=[]
    n=0
    for trackid in track_ids:
        n+=1
        if n % 300 == 0:
            print(f'processed {n} records')

        track_info = sp.track(trackid, market='GB')
        titles.append(track_info['name'])
        artists.append([x['name'] for x in track_info['artists']])
        pp_artist.append(display_artist(track_info))

    return pd.DataFrame({'title': titles, 'artists': artists, 'pp_artist': pp_artist})

In [ ]:
## load track ids AND CLUSTER INFO from csvs

infiles = {'A2': 'Data/nina-staging/batchA2_big_data_id_and_cluster_labels.csv',
            'B': 'Data/nina-staging/batchB_big_data_id_and_cluster_labels.csv',
            'C': 'Data/nina-staging/batchC_big_data_id_and_cluster_labels.csv'}

for k in infiles.keys():
    my_id_labels_df = pd.read_csv(infiles[k], index_col=None)

    artist_etc_df = get_artist_etc_given_id(my_id_labels_df.id.to_list())

    my_track_db = pd.concat([my_id_labels_df, artist_etc_df], axis=1)
    my_track_db.to_csv(f'Data/db/rerun_batch{k}.csv', index=False)
    
print('DONE')

processed 300 records
processed 600 records
processed 900 records
processed 1200 records
processed 1500 records
processed 1800 records
processed 2100 records
processed 2400 records
processed 2700 records
processed 3000 records
processed 3300 records
processed 3600 records
processed 3900 records
processed 4200 records
processed 4500 records
processed 4800 records
processed 5100 records
processed 5400 records
processed 5700 records
processed 6000 records
processed 6300 records
processed 6600 records
processed 6900 records
processed 7200 records
processed 7500 records
processed 7800 records
processed 8100 records
processed 8400 records
processed 8700 records
processed 9000 records
processed 9300 records
processed 9600 records
processed 9900 records


## Print track id, title, and artist to .csv


In [217]:
our_track_db.to_csv('Data/db/nina_day4_tracks.csv', index=False)

### Aside... to debug

bah! this doesn't work right... not sure how to map this function to the series in one shot

In [190]:
def get_track_name_and_combinedartists(trackid):
    """ unused function"""
    track_info = sp.track(trackid, market='GB')
    return track_info['name'], display_artist(track_info)

In [195]:
#test_tracks_df['title'], test_tracks_df['artist'] = 
test_return = test_tracks_df.map(get_track_name_and_combinedartists)

In [197]:
test_return

0                          (Archie, Marry Me, Alvvays)
1                   (Motion Sickness, Phoebe Bridgers)
2                                     (The Wire, HAIM)
3                             (Brazil, Declan McKenna)
4                                 (Hell N Back, Bakar)
5                     (Positive Force, Delicate Steve)
6    (Eat, Sleep, Wake (Nothing But You), Bombay Bi...
7                (Don’t Delete The Kisses, Wolf Alice)
8    (Funeral Singers, Sylvan Esso, Collections Of ...
9                    (Summer Girl - Bonus Track, HAIM)
Name: id, dtype: object

# Generating audio features from Spotify playlists

In [45]:
## house: deep house chill
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX2TRYkJECvfC')

Saved 200 tracks in file: Data/audio_features/37i9dQZF1DX2TRYkJECvfC.csv


In [46]:
## pop: pop all day 
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DXarRysLJmuju')

Saved 100 tracks in file: Data/audio_features/37i9dQZF1DXarRysLJmuju.csv


In [47]:
## pop: big on the internet 
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX5Vy6DFOcx00')

Saved 126 tracks in file: Data/audio_features/37i9dQZF1DX5Vy6DFOcx00.csv


In [48]:
## rock: rock classics https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DWXRqgorJj26U')

Saved 175 tracks in file: Data/audio_features/37i9dQZF1DWXRqgorJj26U.csv


In [49]:
## rock: soft rock https://open.spotify.com/playlist/37i9dQZF1DX6xOPeSOGone 
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX6xOPeSOGone')

Saved 150 tracks in file: Data/audio_features/37i9dQZF1DX6xOPeSOGone.csv


In [50]:
## alternative by decade: 10s https://open.spotify.com/playlist/37i9dQZF1DX873GaRGUmPl
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX873GaRGUmPl')

Saved 80 tracks in file: Data/audio_features/37i9dQZF1DX873GaRGUmPl.csv


In [51]:
## alternative by decade: 00s https://open.spotify.com/playlist/37i9dQZF1DX0YKekzl0blG
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX0YKekzl0blG')

Saved 80 tracks in file: Data/audio_features/37i9dQZF1DX0YKekzl0blG.csv


In [52]:
## alternative by decade: 90s https://open.spotify.com/playlist/37i9dQZF1DXaLI8a395lse
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DXaLI8a395lse')

Saved 80 tracks in file: Data/audio_features/37i9dQZF1DXaLI8a395lse.csv


In [53]:
## alternative by decade: 80s https://open.spotify.com/playlist/37i9dQZF1DWTSKFpOdYF1r
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DWTSKFpOdYF1r')

Saved 80 tracks in file: Data/audio_features/37i9dQZF1DWTSKFpOdYF1r.csv


In [54]:
## soul: feelin good https://open.spotify.com/playlist/37i9dQZF1DX9XIFQuFvzM4
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX9XIFQuFvzM4')

Saved 100 tracks in file: Data/audio_features/37i9dQZF1DX9XIFQuFvzM4.csv


In [55]:
## soul: funk and soul classics https://open.spotify.com/playlist/37i9dQZF1DWWvhKV4FBciw
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DWWvhKV4FBciw')

Saved 80 tracks in file: Data/audio_features/37i9dQZF1DWWvhKV4FBciw.csv


In [56]:
## soul: 70s soul classics https://open.spotify.com/playlist/37i9dQZF1DWULEW2RfoSCi
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DWULEW2RfoSCi')

Saved 91 tracks in file: Data/audio_features/37i9dQZF1DWULEW2RfoSCi.csv


In [57]:
## funk: disco forever https://open.spotify.com/playlist/37i9dQZF1DX1MUPbVKMgJE
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX1MUPbVKMgJE')

Saved 50 tracks in file: Data/audio_features/37i9dQZF1DX1MUPbVKMgJE.csv


In [58]:
## hiphop: hip-hop central https://open.spotify.com/playlist/37i9dQZF1DWY6tYEFs22tT
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DWY6tYEFs22tT')

Saved 76 tracks in file: Data/audio_features/37i9dQZF1DWY6tYEFs22tT.csv


In [59]:
## hiphop: rap caviar https://open.spotify.com/playlist/37i9dQZF1DX0XUsuxWHRQd
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX0XUsuxWHRQd')

Saved 50 tracks in file: Data/audio_features/37i9dQZF1DX0XUsuxWHRQd.csv


In [88]:
## Our morning sunshine guessing game 4ldLuRieh8jnCKOjSoqUQH
save_features_csv_for_all_tracks_in_playlist('4ldLuRieh8jnCKOjSoqUQH')

Saved 16 tracks in file: Data/audio_features/4ldLuRieh8jnCKOjSoqUQH.csv


In [89]:
## House: housewerk 37i9dQZF1DXa8NOEUWPn9W
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DXa8NOEUWPn9W')

Saved 90 tracks in file: Data/audio_features/37i9dQZF1DXa8NOEUWPn9W.csv


In [90]:
## house 37i9dQZF1DWSlskcBvOTt0
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DWSlskcBvOTt0')

Saved 100 tracks in file: Data/audio_features/37i9dQZF1DWSlskcBvOTt0.csv


In [91]:
## house https://open.spotify.com/playlist/37i9dQZF1DWVCHIm2MEeIy
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DWVCHIm2MEeIy')

Saved 70 tracks in file: Data/audio_features/37i9dQZF1DWVCHIm2MEeIy.csv


In [92]:
## 303 classics https://open.spotify.com/playlist/37i9dQZF1DX202yvKb0g9m
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX202yvKb0g9m')

Saved 50 tracks in file: Data/audio_features/37i9dQZF1DX202yvKb0g9m.csv


In [93]:
## acid house https://open.spotify.com/playlist/37i9dQZF1DXbwcS5W73XRr
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DXbwcS5W73XRr')

Saved 48 tracks in file: Data/audio_features/37i9dQZF1DXbwcS5W73XRr.csv


In [94]:
## DJ boring's favorite tracks https://open.spotify.com/playlist/37i9dQZF1DX00RdhV73Dbe
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX00RdhV73Dbe')

Saved 69 tracks in file: Data/audio_features/37i9dQZF1DX00RdhV73Dbe.csv


In [96]:
## https://open.spotify.com/playlist/37i9dQZF1DX8AliSIsGeKd
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX8AliSIsGeKd')

Saved 75 tracks in file: Data/audio_features/37i9dQZF1DX8AliSIsGeKd.csv


In [97]:
## 37i9dQZF1DX0r3x8OtiwEM
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX0r3x8OtiwEM')

Saved 60 tracks in file: Data/audio_features/37i9dQZF1DX0r3x8OtiwEM.csv


In [98]:
## creamy https://open.spotify.com/playlist/37i9dQZF1DXdgz8ZB7c2CP
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX0r3x8OtiwEM')

Saved 60 tracks in file: Data/audio_features/37i9dQZF1DX0r3x8OtiwEM.csv


In [121]:
## techno state https://open.spotify.com/playlist/37i9dQZF1DX8ZMwsPgxIOs
save_features_csv_for_all_tracks_in_playlist('37i9dQZF1DX8ZMwsPgxIOs')

No audio features available for track: 73Wrhs4NIuSXrmY2I2BNFp
Saved 73 tracks in file: Data/audio_features/37i9dQZF1DX8ZMwsPgxIOs.csv


## Loading song titles from our webscraping

In [4]:
fname = 'Data/timeout_best-party-songs.csv'

my_songs = pd.read_csv(fname, index_col=None)

In [60]:
my_songs.head(2)

,artist,title
0,Madonna,Like a Prayer
1,Prince,1999


In [71]:
results = sp.search(q=my_songs.title[0],limit=1,market="GB")

In [78]:
results['tracks']['items'][0].keys()

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [79]:
results['tracks']['items'][0]['id']

'1z3ugFmUKoCzGsI6jdY4Ci'

In [85]:
save_features_csv_for_all_tracks_in_artist_title_csv('Data/timeout_50-best-breakup-songs.csv')

Saved 49 tracks in file: Data/audio_features/timeout_50-best-breakup-songs.csv


In [86]:
save_features_csv_for_all_tracks_in_artist_title_csv('Data/timeout_50-best-sad-songs.csv')
save_features_csv_for_all_tracks_in_artist_title_csv('Data/timeout_best-90s-songs.csv')
save_features_csv_for_all_tracks_in_artist_title_csv('Data/timeout_best-gay-songs.csv')
save_features_csv_for_all_tracks_in_artist_title_csv('Data/timeout_best-happy-songs.csv')
save_features_csv_for_all_tracks_in_artist_title_csv('Data/timeout_best-party-songs.csv')
save_features_csv_for_all_tracks_in_artist_title_csv('Data/timeout_best-pop-songs-of-all-time.csv')
save_features_csv_for_all_tracks_in_artist_title_csv('Data/timeout_the-50-best-80s-songs.csv')
save_features_csv_for_all_tracks_in_artist_title_csv('Data/timeout_the-50-best-karaoke-songs-ever.csv')


Saved 48 tracks in file: Data/audio_features/timeout_50-best-sad-songs.csv
Saved 49 tracks in file: Data/audio_features/timeout_best-90s-songs.csv
Saved 48 tracks in file: Data/audio_features/timeout_best-gay-songs.csv
Saved 24 tracks in file: Data/audio_features/timeout_best-happy-songs.csv
Saved 97 tracks in file: Data/audio_features/timeout_best-party-songs.csv
Saved 40 tracks in file: Data/audio_features/timeout_best-pop-songs-of-all-time.csv
Saved 50 tracks in file: Data/audio_features/timeout_the-50-best-80s-songs.csv
Saved 47 tracks in file: Data/audio_features/timeout_the-50-best-karaoke-songs-ever.csv


In [84]:
def save_features_csv_for_all_tracks_in_artist_title_csv(infilepath):
    
    # extract the name of the input file, without the csv suffix
    songlistname = '.'.join(os.path.basename(infilepath).split('.')[:-1])
    
    outdir = os.path.join('Data', 'audio_features')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    outfilename = os.path.join(outdir, f'{songlistname}.csv')
    
    my_songs_to_find = pd.read_csv(infilepath, index_col=None)
        
    list_of_songs=[]
    for i in range(my_songs_to_find.shape[0]):
        results = sp.search(q=my_songs_to_find.title[i],limit=1,market="GB")
        try:
            list_of_songs.append(sp.audio_features(results['tracks']['items'][0]['id'])[0])
        except:
            print(f'Skipping song due to error: {my_songs_to_find.title[i]}')
            continue

    song_audio_features = pd.DataFrame(list_of_songs)
    song_audio_features.to_csv(outfilename, index=False)
    
    print(f'Saved {my_songs_to_find.shape[0]} tracks in file: {outfilename}')
    return